In [27]:
import numpy as np
from scipy.stats import norm


What is the relationship between `get_trial_likelihood` in `ddm.py` and `get_model_log_likelihood` in `ddm_mla.py`?

In [ ]:
decay = 0.002  # decay = 0 means barriers are constant.
bias = 0
barrier = 1
RT = 2000
timeStep = 10 #ms
approxStateStep = .1
bias = 0
nonDecisionTime = 0
d = .002
sigma = .02
valueLeft = .7
valueRight = .5

In [ ]:
numTimeSteps = RT // timeStep
numTimeSteps

In [ ]:
barrierUp = barrier * np.ones(numTimeSteps)
barrierDown = -barrier * np.ones(numTimeSteps)
for t in range(1, numTimeSteps):
    barrierUp[t] = barrier / (1 + (decay * t))
    barrierDown[t] = -barrier / (1 + (decay * t))
barrierUp

In [ ]:
halfNumStateBins = np.ceil(1 / .1)
halfNumStateBins

In [ ]:
stateStep = 1 / (halfNumStateBins + 0.5)
stateStep

In [ ]:
# The vertical axis is divided into states.
states = np.arange(-1 + (stateStep / 2),
                   1 - (stateStep / 2) + stateStep,
                   stateStep)
states

In [ ]:
biasState = np.argmin(np.absolute(states - bias))

biasState

In [ ]:
# Initial probability for all states is zero, except the bias state,
# for which the initial probability is one.
prStates = np.zeros((states.size, numTimeSteps))
prStates[biasState,0] = 1

# The probability of crossing each barrier over the time of the trial.
probUpCrossing = np.zeros(numTimeSteps)
probDownCrossing = np.zeros(numTimeSteps)

prStates[biasState]

In [ ]:
changeMatrix = np.subtract(states.reshape(states.size, 1), states)
changeUp = np.subtract(barrierUp, states.reshape(states.size, 1))
changeDown = np.subtract(barrierDown, states.reshape(states.size, 1))
changeMatrix[5]

In [ ]:
changeDown[0]

In [ ]:
mean = .0004
sigma = .02
norm.pdf(changeMatrix, mean, sigma)[1]

In [ ]:
mean = .0004
sigma = .02
time= 1

prStatesNew = (stateStep *
               np.dot(norm.pdf(changeMatrix, mean, sigma),
               prStates[:,time-1]))
prStatesNew[(states >= barrierUp[time]) |
            (states <= barrierDown[time])] = 0

prStatesNew

In [ ]:

tempUpCross = np.dot(prStates[:,time-1], (1 - norm.cdf(changeUp[:, time], mean, sigma)))
tempDownCross = np.dot(prStates[:,time-1],norm.cdf(changeDown[:, time], mean, sigma))

tempUpCross

In [ ]:
tempDownCross

In [ ]:
# Renormalize to cope with numerical approximations.
sumIn = np.sum(prStates[:,time-1])
sumCurrent = np.sum(prStatesNew) + tempUpCross + tempDownCross
prStatesNew = prStatesNew * sumIn / sumCurrent
tempUpCross = tempUpCross * sumIn / sumCurrent
tempDownCross = tempDownCross * sumIn / sumCurrent

In [ ]:
sumIn

In [ ]:
sumCurrent

In [ ]:
prStatesNew

In [ ]:
tempUpCross

In [ ]:
tempDownCross

In [ ]:
probDownCrossing[-1] = 2

In [ ]:
probDownCrossing

In [1]:
from ddm import DDMTrial, DDM

In [15]:
m1 = DDM(d = 0.02, sigma = 0.01)

In [16]:
m1_trial = m1.simulate_trial(valueLeft=.8, valueRight=.1)

In [17]:
m1_trial.choice

-1

In [18]:
m1_trial.RT

630

In [19]:
out = m1.get_trial_likelihood(m1_trial)
out['likelihood']

2.1662862883122818e-132

In [21]:
m2 = DDM(d = 0.1, sigma = 0.1)
out2 = m2.get_trial_likelihood(m1_trial)
out2['likelihood']

1.4346374210445043e-06

In [24]:
out2['likelihood'] > out['likelihood']

True

Generate data using one DDM

In [32]:
valueLefts = np.random.uniform(size = 100)
valueRights = np.random.uniform(size = 100)
m1_trials = []

m1 = DDM(d = 0.02, sigma = 0.01)

for valueLeft, valueRight in zip(valueLefts, valueRights): 
    m1_trials.append(m1.simulate_trial(valueLeft=valueLeft, valueRight=valueRight))


Get trial likelihoods for the correct and incorrect DDMs

In [39]:
m1_likelihoods = []
m2_likelihoods = []
m2 = DDM(d = 0.1, sigma = 0.1)

for trial in m1_trials:
    m1_likelihoods.append(m1.get_trial_likelihood(trial)['likelihood'])
    m2_likelihoods.append(m2.get_trial_likelihood(trial)['likelihood'])

Is the sum of the likelihood using the correct model higher than those of the incorrect model?

In [42]:
sum(m1_likelihoods)> sum(m2_likelihoods)

False